In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import colorlover as cl
import random
import shap

from wordcloud import WordCloud
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from plotly.subplots import make_subplots
from nltk.tokenize import TreebankWordTokenizer, TweetTokenizer as twt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec, Doc2Vec, KeyedVectors
from gensim.models.doc2vec import TaggedDocument
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

In [2]:
def process_load_data(dataframes):
    data = pd.concat(dataframes, ignore_index=True)

    # Select columns to be retained and rename them
    selected_columns = [
        'id',
        'context',
        'annotations/0/events/0/event_type',
        'annotations/0/events/0/Trigger/text/0/0',
        'annotations/0/events/0/Treatment/Drug/text/0/0',
        'annotations/0/events/0/Effect/text/0/0'
    ]
    new_names = ['id', 'context', 'event_type', 'trigger_text', 'drug', 'drug_effect']
    df = data[selected_columns].rename(columns=dict(zip(selected_columns, new_names)))

    # Map sentiment labels to numeric values
    sentiment_mapping = {'Adverse_event': 1, 'Potential_therapeutic_event': 0}
    df['sentiment'] = df['event_type'].map(sentiment_mapping)

    return df

In [3]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

dataframes = [train_data, test_data]
df = process_load_data(dataframes)

Columns (112,114,121,125,157,168,186,188,206,208,217,229,231,232,233,235,237,241,244,246,247,249,250,251,268,270,271,280,282,286,288,289,291,305,310,312,313,315,316,318,319,323,324,327,328,329,331,333,335,336,338,342,344,348,350,352,353,355,356,358,359,361) have mixed types. Specify dtype option on import or set low_memory=False.


In [4]:
df.sample(5)

,id,context,event_type,trigger_text,drug,drug_effect,sentiment
2466,8604715_3,Our patient developed both nephrogenic diabete...,Adverse_event,developed,foscarnet,nephrogenic diabetes insipidus and renal tubul...,1
2821,17266059_3,We report a case of Thrombotic Thrombocytopeni...,Adverse_event,to,trimethoprim,Thrombotic Thrombocytopenic Purpura,1
1532,15852680_2,We report a case of interstitial pneumonitis i...,Adverse_event,induced,bicalutamide,interstitial pneumonitis,1
3733,10752888_2,We present two cases of nitrofurantoin-induced...,Adverse_event,induced,nitrofurantoin,pulmonary toxicity in which the initial HRCT s...,1
2609,448845_2,Carbonic anhydrase inhibitors used in the trea...,Potential_therapeutic_event,rarely associated,Carbonic anhydrase inhibitors,blood dyscrasias,0


#### Load pre-trained word vectors from a binary file located at the specified path. The file contains word vectors in a format compatible with Word2Vec. Only the first 100,000 word vectors are loaded.

In [5]:
model_path = './biowordvec/BioWordVec_PubMed_MIMICIII_d200.vec.bin'
model = KeyedVectors.load_word2vec_format(model_path, binary=True, limit=100000)

In [6]:
def average_word_embeddings(df, column, word_embeddings):
    embeddings = []
    for document in df[column]:
        for word in document.split():
            if word in word_embeddings:
                embeddings.append(word_embeddings[word])
    if len(embeddings) > 0:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros_like(word_embeddings.vector_size)

In [7]:
df['average_embeddings'] = df.apply(lambda row: average_word_embeddings(row, 'context', model), axis=1)

In [8]:
df.sample(2)

,id,context,event_type,trigger_text,drug,drug_effect,sentiment,average_embeddings
1432,19745701_1,Increased awareness is needed on the possible ...,Adverse_event,occurrence,MTX,LPD,1,"[0.19952947, 0.13709435, 0.13298526, 0.0314128..."
1824,7416268_1,Disulfiram encephalopathy as a cause of the ca...,Adverse_event,cause,Disulfiram,catatonia syndrome,1,"[0.26490805, 0.16063048, 0.15241699, 0.0170535..."


## MultinomialNB

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('sentiment', axis=1),
                                                    df['sentiment'], test_size=0.2, random_state=42)

scaler = MinMaxScaler(feature_range=(0, 1))

X_train_embeddings = np.array(X_train['average_embeddings'].tolist())
X_test_embeddings = np.array(X_test['average_embeddings'].tolist())

X_train_scaled = scaler.fit_transform(X_train_embeddings)
X_test_scaled = scaler.transform(X_test_embeddings)

pipeline = Pipeline([
    ('scaler', MinMaxScaler(feature_range=(0, 1))),
    ('classifier', MultinomialNB())
])

param_grid = {
    'classifier__alpha': [0.1, 1.0, 10.0],
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train_embeddings, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test_embeddings)

accuracy = accuracy_score(y_test, y_pred) # call cvscore function & pass the entire set CV=5
print("Accuracy:", accuracy)

report = classification_report(y_test, y_pred, zero_division=1)
print("Classification Report:")
print(report)


Accuracy: 0.9095607235142119
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.00      0.00        70
           1       0.91      1.00      0.95       704

    accuracy                           0.91       774
   macro avg       0.95      0.50      0.48       774
weighted avg       0.92      0.91      0.87       774



In [10]:
lines = report.split('\n')
data = [line.split() for line in lines[2:-5]]
columns = ['class', 'precision', 'recall', 'f1-score', 'support']
df_report = pd.DataFrame(data, columns=columns).replace({'class': {'0': 'PTE', '1': 'ADE'}})
df_report

,class,precision,recall,f1-score,support
0,PTE,1.00,0.00,0.00,70
1,ADE,0.91,1.00,0.95,704


### Validate MultinomialNB using a validation dataset ( @Savio discussion point - dev_df is not balanced, negative more than positive 

In [11]:
dev_df = pd.read_csv('data/dev.csv')

validation_dataframe = [dev_df]
df_v = process_load_data(validation_dataframe)

In [12]:
df_v['average_embeddings'] = df_v.apply(lambda row: average_word_embeddings(row, 'context', model), axis=1)

In [13]:
df_v.sample(5)

,id,context,event_type,trigger_text,drug,drug_effect,sentiment,average_embeddings
699,8452107_1,Piritrexim-induced pulmonary toxicity.,Adverse_event,induced,Piritrexim,pulmonary toxicity,1,"[0.29352245, 0.14387782, 0.119526334, -0.02257..."
525,15112258_2,Radiation recall refers to inflammatory reacti...,Adverse_event,triggered,cytotoxic,Radiation recall,1,"[0.23106676, 0.17844391, 0.14350364, 0.0137774..."
112,17959575_16,CONCLUSION: A patient with CHF and ESRD develo...,Adverse_event,after,dobutamine,myoclonic muscle spasms,1,"[0.25916427, 0.14596844, 0.17040022, 0.0197891..."
182,20110001_1,CONCLUSIONS: We present a case of a patient wi...,Adverse_event,developed,imatinib,KS,1,"[0.21228749, 0.22683315, 0.12741666, 0.0952608..."
790,6102186_5,"In addition, loading with one low oral dose of...",Adverse_event,loading,serine,psychotic symptoms,1,"[0.23258963, 0.0965681, 0.15346026, 0.02735474..."


In [14]:
X_dev = df_v.drop('sentiment', axis=1)
y_dev = df_v['sentiment']

X_dev_embeddings = np.array(X_dev['average_embeddings'].tolist())
X_dev_scaled = scaler.transform(X_dev_embeddings)

y_dev_pred = best_model.predict(X_dev_scaled)

accuracy_dev = accuracy_score(y_dev, y_dev_pred)
print("Accuracy on dev set:", accuracy_dev)

report_dev = classification_report(y_dev, y_dev_pred, zero_division=1)
print("Classification Report on dev set:")
print(report_dev)

Accuracy on dev set: 0.8928199791883454
Classification Report on dev set:
              precision    recall  f1-score   support

           0       0.33      0.01      0.02       102
           1       0.89      1.00      0.94       859

    accuracy                           0.89       961
   macro avg       0.61      0.50      0.48       961
weighted avg       0.84      0.89      0.85       961



Here's a breakdown of the evaluation results:

*** For the test set:

- Accuracy: 0.9095607235142119
- Precision, recall, and F1-score for PTE (Potential Therapeutic Event) class (0): The precision is 1.00, recall is 0.00, and F1-score is 0.00. This indicates that the model correctly predicted all instances of ADE (Adverse Event) but failed to predict any instances of PTE.
- Precision, recall, and F1-score for ADE (Adverse Event) class (1): The precision is 0.91, recall is 1.00, and F1-score is 0.95. This indicates that the model performed well in predicting instances of ADE.

*** For the validation set:

- Accuracy: 0.8928199791883454
- Precision, recall, and F1-score for PTE (Potential Therapeutic Event) class (0): The precision is 0.33, recall is 0.01, and F1-score is 0.02. This indicates that the model predicted very few instances of PTE correctly, resulting in low precision, recall, and F1-score for this class.
- Precision, recall, and F1-score for ADE (Adverse Event) class (1): The precision is 0.89, recall is 1.00, and F1-score is 0.94. This indicates that the model performed well in predicting instances of ADE.

Based on the provided classification reports, the model is performing well in predicting instances of Adverse Event (ADE) with high precision, recall, and F1-score. However, the model is not performing well in predicting instances of Potential Therapeutic Event (PTE) as indicated by low precision, recall, and F1-score for the PTE class.

In the test set, the model correctly predicts all instances of ADE but fails to predict any instances of PTE. This is reflected in the high accuracy for the test set. However, it's important to note that the model's inability to predict PTE correctly is a limitation.

In the dev set, the model has low precision, recall, and F1-score for the PTE class, indicating that it struggles to accurately identify instances of PTE. On the other hand, the model performs well in predicting ADE instances, as reflected by the high precision, recall, and F1-score for the ADE class.

Overall, the model's performance is good in predicting ADE but needs improvement in predicting PTE.

## LDA - Incomplete

In [ ]:
X_train_embeddings = X_train.apply(lambda x: average_word_embeddings(x, 'context', word_embeddings))
X_test_embeddings = X_test.apply(lambda x: average_word_embeddings(x, 'context', word_embeddings))

X_train_embeddings = np.stack(X_train_embeddings.values)
X_test_embeddings = np.stack(X_test_embeddings.values)


lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(X_train_embeddings)

X_train_topics = lda.transform(X_train_embeddings)
X_test_topics = lda.transform(X_test_embeddings)


## LDA - Can i combine both ? Is there any value ...